In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
np.random.seed(1337)

import gc
import glob
import cv2
import datetime
import pandas as pd
import time
import scipy
import warnings
warnings.filterwarnings("ignore")
from scipy import misc
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Sequential
from sklearn.model_selection import train_test_split

from models import *
from data_utils import *
from fit_utils import *
from inmem_utils import *
from clr_callback import *

Using TensorFlow backend.


In [ ]:
def fit_model(X, X_tr, X_val, y_tr, y_val, modelname = None, checkname = None, load_name = None):
    callbacks = [ModelCheckpoint('/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/scripts/models/checks/{}.h5'.format(checkname), 
                                        monitor='val_loss', 
                                        verbose = 0, save_best_only = True),
                EarlyStopping(monitor='val_loss', patience = 12, verbose = 1),
                ReduceLROnPlateau(monitor='val_loss', factor = 0.4, verbose = 1, 
                                  patience = 4, min_lr = 5e-6)]
    
    train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.3,
                rotation_range=180,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True,
                vertical_flip=True)
    train_datagen.fit(X, augment = True)
    valid_datagen = ImageDataGenerator(rescale=1./255,)
    
    if 'res' or 'inception' in checkname:
        batch_size = 8
    if 'xception' in checkname:
        batch_size = 4
        
    if load_name is not None:
        model = load_model(checks_src + load_name)
    else:
        model = modelname()
        
    model.fit_generator(train_datagen.flow(X_tr, y_tr, batch_size = batch_size), 
                        steps_per_epoch = X_tr.shape[0]/batch_size,
                        validation_data = valid_datagen.flow(X_val, y_val, batch_size = batch_size, shuffle = False),
                        validation_steps = X_val.shape[0]/batch_size, epochs = 200, callbacks = callbacks)
    
    return model


In [ ]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/training_data/'
checks_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/scripts/models/checks/'
sub_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/submissions/Raw/'
sub_dst = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/submissions/'


In [ ]:
X, _, _ = load_train(src + 'train_crops_frcnn_299/')

X_tr, y_tr, train_ids = load_train(src + 'train_set/')
y_tr = to_categorical(y_tr)

X_val, y_val, train_ids = load_train(src + 'valid_set/')
y_val = to_categorical(y_val)

#X_test, test_ids = load_test(src + 'test_crops_150epochs/test_crops_150epochs/')

In [ ]:
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, Activation
from keras.layers import Input
from keras.models import Model

def sequential_cnn(num_classes, img_size):
    
    model = Sequential()
    x = Conv2D(32, (3, 3), padding='same',
                     input_shape=img_size)(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3)(x))
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2)(x))
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3)(x))
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2)(x))
    x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    output_layer = Activation('softmax')(x)
    
    model = Model(input = input_layer, output = output_layer)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model

def functional_cnn(num_classes, img_size):
    
    input_layer = Input(img_size)
    x = Conv2D(32, (3, 3), padding='same')(input_layer)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    x = Activation('softmax')(x)
    
    model = Model(input = input_layer, output = x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
img_size = (299, 299, 3)
num_classes = 3

model = functional_cnn(num_classes, img_size)
model.fit(X_tr, y_tr, validation_data = (X_val, y_val), batch_size = 8, epochs = 10)

In [ ]:

model = fit_model(X, X_tr, X_val, y_tr, y_val, sequential_cnn, 'sequential_cnn')

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, xception, 'xception_patientsplit_0.2_frcnn170e_crops')

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, xception2, 'xception2_patientsplit_0.2_frcnn170e_crops')

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, inception1, 'inception1_patientsplit_0.3_origdata')

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, inception3, 'inception3_patientsplit_0.3_origdata')

In [ ]:
X_test, test_ids = load_test(src + 'test_crops_150epochs/test_crops_150epochs/')
model = load_model(checks_src + 'resnet1adam_patientsplit_0.2_frcnn170e_crops.h5')
bag_preds = predict_data(model, X_test, 1, 15)
submission_inmem(bag_preds, test_ids, 'xception_patientsplit_origdata')
prep_sub('xception_patientsplit_origdata')